In [142]:
import torch
import pandas as pd
import numpy as np
from generator import *
from discriminator import *
import ipywidgets as widgets
from IPython.display import display
import glob
from sklearn.preprocessing import MinMaxScaler

In [173]:
#交互组件(widget),即按钮、标签、滑块、菜单栏，下拉列表
folder = widgets.Dropdown(
    options=glob.glob("models/dual dis/2/*"),
    description='Folder:',
    disabled=False,
)
display(folder)
print(folder.value)

Dropdown(description='Folder:', options=('models/dual dis/2\\0_ba50_ep500_d3128,64,32_da3256,128,64_g64,128,25…

models/dual dis/2\0_ba50_ep500_d3128,64,32_da3256,128,64_g64,128,256_lrd1e-05_lrda1e-05_lrg5e-05


In [177]:
folder_name = folder.value +"/gen*.pt"
model_widget = widgets.Dropdown(
    options=glob.glob(folder_name),
    description='Generator:',
    disabled=False,
    
)
display(model_widget.value) 

'models/dual dis/2\\4_ba50_ep500_d3128,64,32_da3256,128,64_g64,128,256_lrd1e-05_lrda1e-05_lrg5e-05\\gen.pt'

In [178]:
original_db_path = "data/data_filter_nortar_trun.csv"
original_db = pd.read_csv(original_db_path)
original_db_size = original_db.shape[0]
print(original_db_size)

raw_data = pd.read_csv("data/data_filter.csv")
tar = raw_data[['RT']]
scaler = MinMaxScaler()
a = scaler.fit_transform(tar)

685


In [179]:
import glob
for file in glob.glob(folder_name):
    name = file.split("\\")[-2][:-1]
    print(name)
    checkpoint= torch.load(file, map_location='cpu')
    generator = GeneratorNet(**checkpoint['model_attributes'])
    generator.load_state_dict(checkpoint['model_state_dict'])
    size = original_db_size
    new_data = generator.create_data(size)
    nortar = new_data[:,-1].detach().numpy()
    target = scaler.inverse_transform(np.array(nortar).reshape(-1 ,1))  
    df = pd.DataFrame(np.column_stack((new_data[:,:-1].detach().numpy(), target)), columns=original_db.columns)
    name = name + "_size-" + str(size)
    df.to_csv( "data/fake_data/"+ name + "_trun.csv", index=False)

4_ba50_ep500_d3128,64,32_da3256,128,64_g64,128,256_lrd1e-05_lrda1e-05_lrg5e-0
